### Funções para criação de dados fictícios de votação de evento gastronômico para fins de teste de validação

In [13]:
import random
import time
import names
import lorem
from pyUFbr.baseuf import ufbr
import secrets
import string
import pandas as pd

def gerar_codigos(quantidade, tamanho=6, letras_maiusculas=False, letras_minusculas=True, numeros=True, simbolos=False, prefixo='', contador=0):
    '''Gera códigos aleatórios únicos
    
    Parameters:
        quantidade (inteiro): Quantidade de códigos aleatórios a serem gerados
        tamanho (inteiro): Quantidade de caracteres para cada código
        letras_maiusculas (boolean): Indica se o código irá possuir letras maiúsculas
        letras_minusculas (boolean): Indica se o código irá possuir letras minúsculas
        numeros (boolean): Indica se o código irá possuir números
        simbolos (boolean): Indica se o código irá possuir símbolos
        prefixo (string): prefixo para o código
        contador (inteiro): Contador para a quantidade de chamadas recursivas desta função
    
    Returns:
        Lista de códigos aleatórios gerados ou lista informando para tentar novamente com outros parâmetros
    '''
    
    # Cria a string que define os tipos de caracteres a serem usados no código
    caracteres_codigo = ''
    
    # Verifica os parâmetros dos tipos de caracteres e insere na string    
    if letras_maiusculas:
        caracteres_codigo += string.ascii_uppercase
    if letras_minusculas:
        caracteres_codigo += string.ascii_lowercase
    if numeros:
        caracteres_codigo += string.digits
    if simbolos:
        caracteres_codigo += string.punctuation
    
    # Cria a lista de códigos
    codigos = []
    
    # Itera sobre a quantidade, cria cada código e insere na lista de códigos
    for i in range(quantidade):
        codigo = ''.join(secrets.choice(caracteres_codigo) for i in range(tamanho))
        codigos.append(prefixo + codigo)
    
    # Cria set para verificação da quantidade de códigos únicos
    teste = set(codigos)
    
    # Compara o tamanho do set de teste com a lista de códigos
    # Se algum código não for único ele chama recursivamente a mesma função para tentar de novo
    # Se tentou 10 vezes e não conseguiu gerar a quantidade de códigos únicos, retorna apenas a informação
    if len(teste) != len(codigos):
        if contador < 20:
            # Códigos não são únicos
            contador += 1
            gerar_codigos(quantidade, tamanho, letras_maiusculas, letras_minusculas, numeros, simbolos, prefixo, contador)
        return ['Não foi possível criar a quantidade de códigos aleatório com os parâmetros definidos, tente com maior variedade ou quantidade de caracteres']    
    
    # Se todos os códigos forem únicos, retorna a lista de códigos
    return codigos

def criar_excel(df, caminho='dt_pandas_to_excel.xlsx', indice=True, nome_planilha='Planilha1'):
    '''Cria um arquivo do excel formatado a partir de um data frame pandas
    
    Parameters:
        df (objeto pandas DataFrame): Data frame a ser convertido em planilha do excel
        caminho (string): Caminho do local para salvar o arquivo, com nome do arquivo e extensão
        indice (boolean): Indica se o arquivo irá ou não criar coluna para os índices do data frame
        nome_planilha (string): Nome para a aba/planilha da pasta de trabalho do excel
    
    Returns:
        Não possui retorno    
    '''
    
    # Cria um escritor pandas excel usando XlsxWriter como mecanismo e indicando o caminho como parâmetro
    escritor = pd.ExcelWriter(caminho, engine='xlsxwriter')

    # Converte o data frame em um objeto excel XlsxWriter
    df.to_excel(
        escritor,
        sheet_name=nome_planilha, # Define o nome da aba/planilha da pasta de trabalho
        startrow=1, # Pula uma linha para que seja possível inserir cabeçalho personalizado
        header=False, # Desativa o cabeçalho padrão
        index=indice # Define se irá criar coluna para índice ou não de acordo com o parâmtero informado
    )

    # Obtém a pasta de trabalho XlsxWriter e os objetos de planilha
    pasta_trabalho = escritor.book # Pasta de trabalho
    pasta_trabalho = escritor.sheets[nome_planilha] # Planilha

    # Obtém as dimensões do data frame
    (qtde_linhas, qtde_colunas) = df.shape

    # Cria uma lista de cabeçalhos das colunas para uso no add_table()
    config_colunas = []
    if indice == True: # Se Parâmetro do índice for verdadeiro, inclui o primeiro cabeçalho como 'Índice'
        config_colunas.append({'header': 'Índice'})
    for cabecalho in df.columns: # Para cada coluna no data frame, inclui o cabeçalho na lista
        config_colunas.append({'header': cabecalho})
    
    # Caso parâmetro do índice for falso, diminui uma coluna da quantidade de colunas 
    if indice == False:
        qtde_colunas -= 1
        
    # Adiciona a tabela
    pasta_trabalho.add_table(0, 0, qtde_linhas, qtde_colunas, {'columns': config_colunas})

    # Aumente a largura das colunas para melhor visualização
    pasta_trabalho.set_column(0, qtde_colunas, 25)

    # Fecha o escritor pandas excel writer e salva o arquivo excel
    escritor.save()

def gerar_data(data_inicial='29/07/2021 00:00:00', data_final='12/09/2021 23:59:59', formato='%d/%m/%Y %H:%M:%S', base=random.random()):
    '''Gera data aleatória entre intervalo de datas
    
    Parameters:
        data_inicial (string): Data inicial do intervalo, no formato 'dia/mês/ano horas:minutos'
        data_final (string): Data final do intervalo, no formato 'dia/mês/ano horas:minutos'
        formato (string): Formatação para converter as strings de data para objeto time
        base (método): método do objeto random para gerar valores aleatórios
    
    Returns:
        String com a data aleatória formatada entre os intervalos
    '''
    
    # Converte a string da data inicial em objeto time
    time_data_inicial = time.mktime(time.strptime(data_inicial, formato))
    
    # Converte a string da data final em objeto time
    time_data_final = time.mktime(time.strptime(data_final, formato))
    
    # Gera a data aleatória entre o intervalo
    data_aleatoria = time_data_inicial + base * (time_data_final - time_data_inicial)
    
    # Retorna a data aleatória
    return time.strftime(formato, time.localtime(data_aleatoria))

def gerar_nota(nota_inicial=5, nota_final=10):
    '''Gera nota aleatória entre intervalo de notas
    
    Parameters:
        nota_inicial (inteiro): Nota inicial
        nota_final (inteiro): Nota final
    
    Returns:
        Inteiro, nota aleatória
    '''
    
    # Retorna nota aleatória
    return random.randrange(nota_inicial, nota_final+1)

def gerar_cidade():
    '''Gera cidade brasileira aleatória
    
    Parameters:
        nenhum
    
    Returns:
        Tupla contendo cidade, UF
    '''
    
    # Seleciona unidade federativa aleatória
    uf = random.choice(ufbr.list_uf)
    
    # Seleciona município aleatório da unidade federativa escolhida aleatoriamente
    cidade = random.choice(ufbr.list_cidades(uf))
    
    return cidade, uf

def obter_codigo(df):
    '''Obter código a partir de um data frame pandas de códigos
    
    Parameters:
        df (pandas data frame): Data Frame pandas contendo os códigos, onde os códigos devem estar na primeira coluna
    
    Returns:
        String contendo o código
    '''

    # Estabelece um índice aleatório dentre todos os registros do data frame
    indice = random.randrange(0, df[df.columns[0]].nunique())
    
    # Obtém o código referente ao índice aleatório
    codigo = df.loc[indice, df.columns[0]]
    
    # Retorna o código obtido aleatoriamente do data frame
    return codigo

def criar_voto_ficticio(aleatorio=True, voto_fake=False, **kwargs):
    '''Cria um dicionário com dados fictícios de votação para fins de teste
    
    Parameters:
        aleatorio (boolean): Indica se os dados serão aleatórios ou não
        voto_fake (boolean): Indica se o código a ser inserido no teste poderá ser falso
        df (pandas data frame): Data frame contendo todos os códigos gerados para inclusão de um deles no voto fictício
        data (string): Data formatada para inserir no voto fictício para o caso de não ser aleatório
        codigo (string): Código para inserir no voto fictício para o caso de não ser aleatório
        cupom (string): Cupom fiscal para inserir no voto fictício para o caso de não ser aleatório
        nota1 (inteiro): Nota de 5 a 10 referente à originalidade para inserir no voto fictício para o caso de não ser aleatório
        nota2 (inteiro): Nota de 5 a 10 referente ao sabor do prato para inserir no voto fictício para o caso de não ser aleatório
        nota3 (inteiro): Nota de 5 a 10 referente à apresentação para inserir no voto fictício para o caso de não ser aleatório
        nota4 (inteiro): Nota de 5 a 10 referente à criatividade para inserir no voto fictício para o caso de não ser aleatório
        nota5 (inteiro): Nota de 5 a 10 referente ao atendimento para inserir no voto fictício para o caso de não ser aleatório
        nome (string): Nome para inserir no voto fictício para o caso de não ser aleatório
        telefone (string): Telefone para inserir no voto fictício para o caso de não ser aleatório
        email (string): Email para inserir no voto fictício para o caso de não ser aleatório
        cidade (string): Cidade para inserir no voto fictício para o caso de não ser aleatório
        uf (string): Unidade Federativa para inserir no voto fictício para o caso de não ser aleatório
        soube_evento (string): Informação sobre como conheceu o evento para inserir no voto fictício para o caso de não ser aleatório
        
    Returns:
        Dicionário de voto fictício
    '''
    
    # Define o dicionário para teste
    dicionario_voto = {}
    
    # Cria um voto caso a opção de voto aleatório seja verdadeira
    if aleatorio == True:
        # Cria nome completo aleatório
        nome = names.get_full_name()
        # Cria cidade brasileira aleatória
        cidade = gerar_cidade()
        # Adiciona os dados aleatório ao voto fictício
        dicionario_voto['Carimbo de data/hora'] = gerar_data()
        # Em caso de possibilidade de voto fake, cria um código aleatório
        if voto_fake == True:
            dicionario_voto['Código de Validação'] = gerar_codigos(1)[0]
        else:
            dicionario_voto['Código de Validação'] = obter_codigo(kwargs['df'])
        dicionario_voto['Número do Cupom Fiscal'] = gerar_codigos(1, letras_minusculas=False)[0]
        dicionario_voto['Originalidade'] = gerar_nota()
        dicionario_voto['Sabor do prato'] = gerar_nota()
        dicionario_voto['Apresentação'] = gerar_nota()
        dicionario_voto['Criatividade'] = gerar_nota()
        dicionario_voto['Atendimento'] = gerar_nota()
        dicionario_voto['Nome do cliente'] = nome
        dicionario_voto['Telefone com DDD'] = gerar_codigos(1, tamanho=11, letras_minusculas=False)[0]
        dicionario_voto['Email'] = f'{nome.split(" ")[0]}@emailteste.com'
        dicionario_voto['Cidade'] = cidade[0]
        dicionario_voto['Estado'] = cidade[1]
        dicionario_voto['Como soube do evento?'] = lorem.sentence()
    else:
        dicionario_voto['Carimbo de data/hora'] = kwargs['data']
        dicionario_voto['Código de Validação'] = kwargs['codigo']
        dicionario_voto['Número do Cupom Fiscal'] = kwargs['cupom']
        dicionario_voto['Originalidade'] = kwargs['nota1']
        dicionario_voto['Sabor do prato'] = kwargs['nota2']
        dicionario_voto['Apresentação'] = kwargs['nota3']
        dicionario_voto['Criatividade'] = kwargs['nota4']
        dicionario_voto['Atendimento'] = kwargs['nota5']
        dicionario_voto['Nome do cliente'] = kwargs['nome']
        dicionario_voto['Telefone com DDD'] = kwargs['telefone']
        dicionario_voto['Email'] = kwargs['email']
        dicionario_voto['Cidade'] = kwargs['cidade']
        dicionario_voto['Estado'] = kwargs['uf']
        dicionario_voto['Como soube do evento?'] = kwargs['soube_evento']
    
    # Retorna o dicionário de voto fictício
    return dicionario_voto

### Criação de dados fictícios de votação de evento gastronômico para fins de teste de validação

In [11]:
import pandas as pd

# Abre uma planilha do excel com códigos de validação como data frame
# Atribui ao mesmo data frame apenas sua coluna de códigos
df_codigos = pd.read_excel('codigos-categorias-geral.xlsx')
df_codigos = pd.DataFrame(df_codigos['Código'])

# Cria lista de votos fictícios para teste
votos_teste = []

# Adiciona voto fictício manualmente
votos_teste.append(criar_voto_ficticio(
    aleatorio=False,
    data = '22/08/2021 18:33:01',
    codigo = 'd56fsa',
    cupom = '15836',
    nota1 = 7,
    nota2 = 8,
    nota3 = 9,
    nota4 = 7,
    nota5 = 9,
    nome = 'Hiago',
    telefone = '12999999999',
    email = 'teste@teste.com',
    cidade = 'Caraguatatuba',
    uf = 'SP',
    soube_evento = 'Internet'
))

# Adiciona 5 votos aleatórios com códigos reais
for i in range(5):
    votos_teste.append(criar_voto_ficticio(df=df_codigos))

# Adiciona 3 votos aleatórios com códigos recém criados e que podem não constar na planilha de códigos
# Considerando como voto falso
for i in range(3):
    votos_teste.append(criar_voto_ficticio(voto_fake=True, df=df_codigos))
    
# Adiciona 3 votos aleatórios com códigos reais
for i in range(3):
    votos_teste.append(criar_voto_ficticio(df=df_codigos))
    
# Cria data frame com os votos fictícios 
df_votos_teste = pd.DataFrame(votos_teste)

# Cria pasta de trabalho do excel com o data frame
criar_excel(df_votos_teste, caminho='votos-teste.xlsx', indice=False)

### Preenchendo tabela de códigos com dados fictícios de estabelecimento

In [ ]:
def gerar_estabelecimentos(quantidade):
    '''Retorna nomes aleatórios para estabelecimento
    
    Parameters:
        quantidade (inteiro): quantidade de nomes aleatórios a serem gerados
            
    Returns:
        Lista com nomes de estabelecimento
    '''
    import names
    
    estabelecimentos = [(names.get_full_name() + ' LTDA') for i in range(quantidade)]
    
    return estabelecimentos

def estabelecimento_aleatorio(categoria, dic_estabelecimentos):
    '''Retorna um estabelecimento aleatório de uma categoria a partir de um dicionário contendo listas de estabelecimento por categoria
    
    Parameters:
        categoria (string): Categoria do estabelecimento a ser retornado
        dic_categorias (dicionario): Dicionário com as categorias como chave e listas de estabelecimento como valores
    
    Returns:
        String com nome do estabelecimento
    '''
    return random.choice(dic_estabelecimentos[categoria])

# Lista de Categorias
categorias = [
    'Café',
    'Carnes e Aves',
    'Comida de Boteco ',
    'Confeitaria',
    'Culinária Internacional',
    'Frutos do Mar',
    'Lanches Artesanais',
    'Pizzas',
    'Sabor de Praia',
    'Sobremesas Frias',
]

# Criar dicionário com estabelecimentos aleatórios por categoria
dic_estabelecimentos = {}
for categoria in categorias:
    dic_estabelecimentos[categoria] = gerar_estabelecimentos(5)

# Abre uma planilha do excel com códigos de validação como data frame
df_codigos = pd.read_excel('codigos-categorias-geral.xlsx')

# Obtendo quantidade de códigos do data frame
tamanho_df_codigos = int(df_codigos.nunique()[0])

# Atribuindo um estabelecimento aleatório aos códigos para testar
for i in range(tamanho_df_codigos):
    df_codigos.loc[i, 'Estabelecimento'] = estabelecimento_aleatorio(df_codigos.loc[i, 'Categoria'], dic_estabelecimentos)
    
# Ordenando o data frame por categoria e estabelecimento
df_codigos = df_codigos.sort_values(['Categoria', 'Estabelecimento'])